In [23]:
import os
import pandas as pd
from io import StringIO

# Data Preprocessing

## Polls Aggregation

In [24]:
def process_year_state(year, state, cutoff_date):
    # Parse the HTML file and extract the table
    html_literal = open(f'./{year}/{state}.txt').read()
    
    # Create and preprocess the DataFrame
    df = pd.read_html(StringIO(html_literal))[0].iloc[:, :6]
    df.columns = df.columns.str.replace(r'.*\(D\).*', 'DEM', regex=True)
    df.columns = df.columns.str.replace(r'.*\(R\).*', 'REP', regex=True)
    cols_to_convert = ['REP', 'DEM', 'moe']
    df['moe'] = df['moe'].str.replace('—', 'NaN')
    df[cols_to_convert] = df[cols_to_convert].astype(float)
    df['moe'] = df['moe'].fillna(df['moe'].median())
    df['spread'] = df['REP'] - df['DEM']
    
    # Specific slicing based on year
    if year != 2024:
        df['real_spread'] = df.iloc[0]['REP'] - df.iloc[0]['DEM']
        df = df.iloc[2:].reset_index(drop=True)
    else:
        df = df.iloc[1:].reset_index(drop=True)

    # Extract sample size and respondent type
    df[['sample_size', 'respondent_type']] = df['sample'].str.extract(r'(?:(\d+)\s+)?(LV|RV)')
    df['sample_size'] = pd.to_numeric(df['sample_size'], errors='coerce')
    df['sample_size'] = df['sample_size'].fillna(df['sample_size'].median()).astype(int)
    
    # Process start and end dates
    df[['start_date', 'end_date']] = df['date'].str.split(' - ', expand=True)
    df['start_date'] = df['start_date'] + f'/{year}'
    df['end_date'] = df['end_date'] + f'/{year}'
    df['start_date'] = pd.to_datetime(df['start_date'], format='%m/%d/%Y')
    df['end_date'] = pd.to_datetime(df['end_date'], format='%m/%d/%Y')
    
    # Filter based on cutoff date
    df['state'] = state
    df['pollster'] = df['pollster'].str.replace('*', '').fillna('').str.replace(r'\/.*', '', regex=True).str.strip()
    df['pollster'] = df['pollster'].str.replace('NBC News', 'NBC')
    
    # Ensure the directory exists and save the CSV
    os.makedirs(f'./{year}/csv', exist_ok=True)
    if year == 2024:
        df[['pollster', 'start_date', 'end_date', 'state', 'sample_size', 'respondent_type', 'REP', 'DEM', 'moe', 'spread']].to_csv(f'./{year}/csv/{state}.csv', index=False)
    else:
        df[['pollster', 'start_date', 'end_date', 'state', 'sample_size', 'respondent_type', 'REP', 'DEM', 'moe', 'spread', 'real_spread']].to_csv(f'./{year}/csv/{state}.csv', index=False)
        
# Process all years and states
years = [2016, 2020, 2024]
swing_states = ['AZ', 'GA', 'MI', 'NC', 'NV', 'PA', 'WI']
cutoff_date = "10-01"
for year in years:
    for state in swing_states:
        process_year_state(year, state, cutoff_date)


In [25]:
data_h = pd.DataFrame()

for year in [2016, 2020]:
    for state in swing_states:
        df = pd.read_csv(f'./{year}/csv/{state}.csv')
        data_h = pd.concat([data_h, df])

data_24 = pd.DataFrame()

for state in swing_states:
    df = pd.read_csv(f'./2024/csv/{state}.csv')
    data_24 = pd.concat([data_24, df])

data_h.to_csv('./2016_2020.csv', index=False)
data_24.to_csv('./2024.csv', index=False)

## Pollster Bias Determination by ChatGPT

In [26]:
pollster_union = pd.concat([data_h['pollster'], data_24['pollster']]).drop_duplicates().sort_values().to_numpy()

prompt = f"""Hi, I am currently doing pollster data preparation and I need you to map the pollster names to their political preferences objectively.

I need two scales:
1. "pro-democratic", "independent", "pro-republican"
2. "strongly pro-republican", "likely pro-republican", "independent", "likely pro-democratic", "strongly pro-democratic"

Here are the pollster names: {pollster_union}

Return the mapping as two maps: `3_scale_bias` and `5_scale_bias`."""

print(prompt)

Hi, I am currently doing pollster data preparation and I need you to map the pollster names to their political preferences objectively.

I need two scales:
1. "pro-democratic", "independent", "pro-republican"
2. "strongly pro-republican", "likely pro-republican", "independent", "likely pro-democratic", "strongly pro-democratic"

Here are the pollster names: ['8 News NOW - Las Vegas' 'AmGreatness' 'Atlas Intel' 'Bloomberg'
 'CBS News' 'CNBC' 'CNN' 'Data Orbital' 'Detroit Free Press'
 'East Carolina U.' 'Emerson' 'FOX 2 Detroit' 'FOX News' 'Fabrizio'
 'Gravis' 'Harper (R)' 'InsiderAdvantage' 'Landmark' 'Loras' 'MNS'
 'Marist' 'Marquette' 'Mitchell Research' 'Monmouth' 'Morning Call'
 'Morning Consult' 'Muhlenberg College' 'NBC' 'NY Times'
 'Noble Predictive Insights' 'Opinion Savvy' 'PPP (D)' 'Quinnipiac'
 'Rasmussen Reports' 'Remington Research (R)' 'Reuters' 'Susquehanna'
 'The Hill' 'Trafalgar Group (R)' 'UMass Lowell' 'USA Today' 'WRAL-TV'
 'WSB-TV' 'Washington Post']

Return the map

In [27]:
three_scale_bias = {
    '8 News NOW - Las Vegas': 'independent',
    'ABC News': 'pro-democratic',
    'AmGreatness': 'pro-republican',
    'Arizona Republic': 'independent',
    'Atlanta Journal-Constitution': 'pro-democratic',
    'Atlas Intel': 'independent',
    'Bloomberg': 'pro-democratic',
    'CBS News': 'pro-democratic',
    'CNBC': 'pro-democratic',
    'CNN': 'pro-democratic',
    'Carolina Journal': 'pro-republican',
    'Change Research (D)': 'pro-democratic',
    'Civitas': 'independent',
    'Civitas (R)': 'pro-republican',
    'Cook Political Report': 'independent',
    'Data Orbital': 'pro-republican',
    'Detroit Free Press': 'independent',
    'Detroit News': 'independent',
    'EPIC-MRA': 'independent',
    'East Carolina U.': 'independent',
    'Elon': 'independent',
    'Elon University': 'independent',
    'Emerson': 'independent',
    'FOX 2 Detroit': 'pro-republican',
    'FOX 5': 'pro-republican',
    'FOX 5 Atlanta': 'pro-republican',
    'FOX News': 'pro-republican',
    'Fabrizio': 'pro-republican',
    'Franklin & Marshall': 'independent',
    'GSG': 'pro-democratic',
    'Gravis': 'independent',
    'Harper (R)': 'pro-republican',
    'High Point': 'independent',
    'High Point University': 'independent',
    'InsiderAdvantage': 'pro-republican',
    'JMC Analytics': 'pro-republican',
    'KPHO-TV': 'independent',
    'KTNV': 'independent',
    'Landmark': 'pro-republican',
    'Landmark Communications': 'pro-republican',
    'Las Vegas Review-Journal': 'pro-republican',
    'Loras': 'independent',
    'MIRS': 'independent',
    'MIRS-GSCI': 'independent',
    'MNS': 'independent',
    'MRG': 'independent',
    'Marist': 'independent',
    'Marquette': 'independent',
    'Mason-Dixon': 'independent',
    'Mercyhurst': 'independent',
    'Mercyhurst University': 'independent',
    'Mitchell Research': 'independent',
    'Monmouth': 'pro-democratic',
    'Morning Call': 'independent',
    'Muhlenberg College': 'independent',
    'NBC': 'pro-democratic',
    'NY Times': 'pro-democratic',
    'Noble Predictive Insights': 'independent',
    'OAN': 'pro-republican',
    'OH Predictive Insights': 'independent',
    'Opinion Savvy': 'independent',
    'PPP (D)': 'pro-democratic',
    'PollingPlus': 'independent',
    'Quinnipiac': 'pro-democratic',
    'Rasmussen Reports': 'pro-republican',
    'Remington Research (R)': 'pro-republican',
    'Reuters': 'independent',
    'Suffolk': 'independent',
    'Suffolk University': 'independent',
    'SurveyUSA': 'independent',
    'Susquehanna': 'pro-republican',
    'TIPP': 'independent',
    'The Hill': 'independent',
    'Trafalgar Group (R)': 'pro-republican',
    'U. of Wisconsin': 'independent',
    'UMass Lowell': 'independent',
    'UNLV': 'independent',
    'USA Today': 'independent',
    'Univ. of Georgia': 'independent',
    'Univ. of Wis': 'independent',
    'Univision': 'pro-democratic',
    'Victory Insights': 'independent',
    'WPR': 'independent',
    'WRAL-TV': 'pro-democratic',
    'WSB-TV': 'independent',
    'WXIA-TV': 'independent',
    'Wall Street Journal': 'independent',
    'Washington Post': 'pro-democratic',
    'Yahoo News': 'pro-democratic',
    'co': 'independent'
}

five_scale_bias = {
    '8 News NOW - Las Vegas': 'independent',
    'ABC News': 'likely pro-democratic',
    'AmGreatness': 'strongly pro-republican',
    'Arizona Republic': 'independent',
    'Atlanta Journal-Constitution': 'likely pro-democratic',
    'Atlas Intel': 'independent',
    'Bloomberg': 'likely pro-democratic',
    'CBS News': 'likely pro-democratic',
    'CNBC': 'likely pro-democratic',
    'CNN': 'strongly pro-democratic',
    'Carolina Journal': 'likely pro-republican',
    'Change Research (D)': 'strongly pro-democratic',
    'Civitas': 'independent',
    'Civitas (R)': 'likely pro-republican',
    'Cook Political Report': 'independent',
    'Data Orbital': 'likely pro-republican',
    'Detroit Free Press': 'independent',
    'Detroit News': 'independent',
    'EPIC-MRA': 'independent',
    'East Carolina U.': 'independent',
    'Elon': 'independent',
    'Elon University': 'independent',
    'Emerson': 'independent',
    'FOX 2 Detroit': 'likely pro-republican',
    'FOX 5': 'likely pro-republican',
    'FOX 5 Atlanta': 'likely pro-republican',
    'FOX News': 'strongly pro-republican',
    'Fabrizio': 'likely pro-republican',
    'Franklin & Marshall': 'independent',
    'GSG': 'strongly pro-democratic',
    'Gravis': 'independent',
    'Harper (R)': 'likely pro-republican',
    'High Point': 'independent',
    'High Point University': 'independent',
    'InsiderAdvantage': 'likely pro-republican',
    'JMC Analytics': 'likely pro-republican',
    'KPHO-TV': 'independent',
    'KTNV': 'independent',
    'Landmark': 'likely pro-republican',
    'Landmark Communications': 'likely pro-republican',
    'Las Vegas Review-Journal': 'likely pro-republican',
    'Loras': 'independent',
    'MIRS': 'independent',
    'MIRS-GSCI': 'independent',
    'MNS': 'independent',
    'MRG': 'independent',
    'Marist': 'independent',
    'Marquette': 'independent',
    'Mason-Dixon': 'independent',
    'Mercyhurst': 'independent',
    'Mercyhurst University': 'independent',
    'Mitchell Research': 'independent',
    'Monmouth': 'likely pro-democratic',
    'Morning Call': 'independent',
    'Muhlenberg College': 'independent',
    'NBC': 'likely pro-democratic',
    'NY Times': 'strongly pro-democratic',
    'Noble Predictive Insights': 'independent',
    'OAN': 'strongly pro-republican',
    'OH Predictive Insights': 'independent',
    'Opinion Savvy': 'independent',
    'PPP (D)': 'strongly pro-democratic',
    'PollingPlus': 'independent',
    'Quinnipiac': 'likely pro-democratic',
    'Rasmussen Reports': 'likely pro-republican',
    'Remington Research (R)': 'likely pro-republican',
    'Reuters': 'independent',
    'Suffolk': 'independent',
    'Suffolk University': 'independent',
    'SurveyUSA': 'independent',
    'Susquehanna': 'likely pro-republican',
    'TIPP': 'independent',
    'The Hill': 'independent',
    'Trafalgar Group (R)': 'strongly pro-republican',
    'U. of Wisconsin': 'independent',
    'UMass Lowell': 'independent',
    'UNLV': 'independent',
    'USA Today': 'independent',
    'Univ. of Georgia': 'independent',
    'Univ. of Wis': 'independent',
    'Univision': 'likely pro-democratic',
    'Victory Insights': 'independent',
    'WPR': 'independent',
    'WRAL-TV': 'likely pro-democratic',
    'WSB-TV': 'independent',
    'WXIA-TV': 'independent',
    'Wall Street Journal': 'independent',
    'Washington Post': 'strongly pro-democratic',
    'Yahoo News': 'likely pro-democratic',
    'co': 'independent'
}


In [28]:
pollster_3_scale_bias_num = {
    'pro-republican': 1,
    'independent': 0,
    'pro-democratic': -1
}

pollster_5_scale_bias_num = {
    'strongly pro-republican': 2,
    'likely pro-republican': 1,
    'independent': 0,
    'likely pro-democratic': -1,
    'strongly pro-democratic': -2
}

respondent_type_numeric = {
    'RV': 0,
    'LV': 1
}

## Mapping Bias & Converting Date and Respondent Type to numeric Format

In [29]:

data_h['respondent_type'] = data_h['respondent_type'].fillna('RV')
data_24['respondent_type'] = data_24['respondent_type'].fillna('RV')

data_h['3_scale_bias'] = data_h['pollster'].map(three_scale_bias)
data_h['5_scale_bias'] = data_h['pollster'].map(five_scale_bias)

data_h['3_scale_bias_num'] = data_h['3_scale_bias'].map(pollster_3_scale_bias_num)
data_h['5_scale_bias_num'] = data_h['5_scale_bias'].map(pollster_5_scale_bias_num)

data_h['respondent_type_num'] = data_h['respondent_type'].map(respondent_type_numeric)

data_24['3_scale_bias'] = data_24['pollster'].map(three_scale_bias)
data_24['5_scale_bias'] = data_24['pollster'].map(five_scale_bias)
data_24['3_scale_bias_num'] = data_24['3_scale_bias'].map(pollster_3_scale_bias_num)
data_24['5_scale_bias_num'] = data_24['5_scale_bias'].map(pollster_5_scale_bias_num)
data_24['respondent_type_num'] = data_24['respondent_type'].map(respondent_type_numeric)


# encode dates as numeric
data_h['start_date'] = pd.to_datetime(data_h['start_date'])
data_h['end_date'] = pd.to_datetime(data_h['end_date'])

data_h['year'] =  data_h['end_date'].dt.year
data_h['month_start'] =  data_h['start_date'].dt.month
data_h['day_start'] =  data_h['start_date'].dt.day
data_h['month_end'] =  data_h['end_date'].dt.month
data_h['day_end'] =  data_h['end_date'].dt.day

data_24['start_date'] = pd.to_datetime(data_24['start_date'])
data_24['end_date'] = pd.to_datetime(data_24['end_date'])

data_24['year'] =  data_24['end_date'].dt.year
data_24['month_start'] =  data_24['start_date'].dt.month
data_24['day_start'] =  data_24['start_date'].dt.day
data_24['month_end'] =  data_24['end_date'].dt.month
data_24['day_end'] =  data_24['end_date'].dt.day

data_h.to_csv('./2016_2020.csv', index=False)
data_24.to_csv('./2024.csv', index=False)